# Report Notes

## TODO:
* describe how equal sign is calculated in segmenatation part.
* in introduction add note that we assume the reader has a background in theory of neural networks and geometry

## Introduction

With computational resources and storage getting cheaper and cheaper, a window of possibilities opens for searching, reusing, and processing information. Thus everything around gets digitized, including all kind of paper documentation. And till now most of that work is done manually, which is extremely expensive. And for mathematical papers specifically, it becomes even more time-consuming, since it's far harder to type a mathematical formula rather than a regular word. 

For that reason, hundred and hundred years of math history is still stored in paper format exclusively and cannot be processed efficiently. And while we already have comprehensive tools to parse scanned text [TESSERACT](), but it's not applicable to math expressions, which have a big complexity for parsing. Although there are some decent tools[SCANNER LATEX](), they are proprietary and have limited functionality, in particular, it cannot parse multiline expressions or even entire documents. For that reason, we will be working on an open-source project to parse math expressions so that it could be freely reused and help developing tools, which can convert the entire math paper at once. Only then we will have a dramatic effect of digitizing all math literature.

But since it's a project in the field of OCR, which is a good way to start for newcomers in Computer Vision, we decided to describe our project in self-contained and educative manner. That will allow the reader to enter the field of CV smoothly by exploring and playing with our project. We will also encourage readers to contribute to the project, which they will know by heart until the end of the paper.  

## Problem Definition

Right now, Optical Character Recognition is a solved task for a regular text. We have plenty of approaches and available solution to parse entire files with either handwritten or scanned typed text.

But for some specific class of documents, such as math papers, the problem still exists. The main complexity here is that parsing math expression is untrivial, since, depending on the context, some character will overlap other, appear on the top of each other, or even span multiple lines. Thus we want to create an open-source project tackling this problem so that it could be reused by a community and eventually move us a little closer to the destination of the document-wise parser of math papers.

## Algorithm. Character recognition

For character recognition, the most frequently used tool is a neural network. (Add why! e.g. "since they are the best at recognizing the patterns?"). And we will use a basic version of neural network popular variation - convolutional neural network. If you are not familiar with the basic theory of CNN, we recommend watching this material [LINK]() and also any other you will find on the Internet. But in any case, below, we will give a very high-level explanation of how it works better with computer vision than a classical neural network.

So in the classical neural network, we have an input and output vector. Our input will be the flattened image, that is if we have a 28x28 image represented as a matrix, we can also express it as a vector with 784 elements, where first 28 items equivalent to the first row of a matrix, the second 28 items - to the second row, etc. But with this approach, we are not enforcing our model to work on the level of some groups of related pixels but rather as an independent set of 784 input signals. While in real life, a pixel just above or below has a very strong relationship with each other, while those influence far lower with pixels in another part of an image.

And convolutional neural network solves this issue by working one level higher than merely on the pixel scale. It transforms the image by scanning it with a rectangular kernel, thus learning from the patterns on a level of that rectangular window sliding through the image. It also can reduce the dimension of input image following some rule, e.g., replaces 2x2 part of the image with a single pixel, which value equals the maximum from 4 pixels in the original image. After that, we can scan the picture again with a rectangular window and learn some more high-level features.

Convolutions allow us to learn a pattern depending on the geographical location of images. And pooling enables us to get rid of some local and specific pattern and work with something more high level as corners and lines of an image.

### Our implementation
In our implementation we used the same ideas. The first is the convolutional layer, which is like a set of learnable filters. There are 32 filters for the two firsts convolutional layers, and 64 filters for the two last ones. Each filter transforms a part of the image (defined by the kernel size) using the kernel filter. 

It is followed by a pooling layer, which decreases the dimension of the image. These are used to reduce the computational cost and, to some extent, also minimize overfitting. Moreover, it's useful to allow consequent convolutional layers to learn more high-level features.

Also we have dropout layers, which are used to randomly ignore some fraction of output nodes from previous layers. This way we force the network to learn features in a distributed way, which, in turn, improves generalization and reduces the overfitting.

And at the end, we flatten our final feature maps into a one-dimensional vector, which will combine all founded local and global features. And then based on that feature vector, we output a vector where i-th elements is the probability of i-th character being on the input picture

During this process, we also to standard improvements to the quality of the model and the training time. For example, we can decrease the latter by using a dynamically-changed learning rate. In the beginning it will be bigger, and when the precision of the model doesn't increase significantly, we'll decrease the learning rate by a factor of 2. What concerns quality enhancements, here we incorporate data augmentation as well. Although, we only use a little zooming, horizontal and vertical shifts, and a small-angle rotation so that not corrupt the image. That is, if we applied vertical flipping to "6" symbol, we'd get a "9" symbol with the old label. Or if we rotate "l" too much to the left, we will get something more similar to division sign "/". All those confusions will decrease the precision of the network.

## Future Improvements

Since the topic has a lot of complications, there are multiples possibilities to improve the solution. Currently, it's working in its basic form, that is parsing a single-line formula with brackets, subtraction, addition, multiplication, and division.

The next improvement should be the ability to parse subscript and superscript properly. As a functionality which correctly identifies "=" sign, that extension requires only pure Computer Science enhancements. That is, we already have a feature, which determines the bounding box for each character. Now if we draw a horizontal line in the middle of a total bounding box, we could identify sub- and subscript. That is when a bounding box is crossed by that line; then it's a pure character. Otherwise, it's superscript or subscript character when bounding box is above or below the line respectively.

Besides that, there plenty other more complicated features that can be added. Starting from something relatively simple as identifying a fraction, where we could also use a calculation based on bounding boxes, or parsing trigonometric operators to something more complex such as $\sum_{i=1}^{n}$, or multiline system of linear equations.

We could also improve the quality of existing functionality. While formula segmentation works precisely, we can extend it to work with formulas in the wild(whiteboard, street art, etc.) as well. And what concerns character recognition module, here we have a huge room for improvement, since its precision is only 88%

## Conclusion

So in this work we tackled the problem of optical character recognition in the math domain. To do so, we incorporated two fundamental techniques of that field: character recognition and string segmentation. Former gave us experience in applying classic machine learning techniques with CNN, while in latter we used a lot of graphical algorithms from computer science field. But the most efforts, improvements, and troubleshooting were dedicated to data preprocessing part. Since not only we were required to adjust the dataset to a specific need of our project, but also merge two different datasets, which opened us another huge task of adjusting one dataset format to another.

As for someone with no prior ML experience, this project was extremely valuable in learning basic but vital parts of most computer vision problem. It gave us a broad experience of computer vision tools: preprocessing of image database, graphical algorithms, and neural network. And that is the reason, why we decided to format this report in a self-containing and educative manner. Since we grabbed so broad and fundamental experience, we wanted to share it to other newcomers into ML and CV in particular. Something that would have been useful to us at the beginning of the journey.

Besides that, we are going to continue the work on this project, which will give us the ability to create a second part of the tutorial, and thus we will produce something more applicable to real-life problems. In that way, our solution will be useful not only as a learning example or repository where anybody could reuse some of the basic functionality for OCR, but also will be valuable as a complete solution on its own.